# Querying the dynamical.org STAC Catalog with rustac

This notebook shows how to query the dynamical.org STAC catalog stored as a
[stac-geoparquet](https://github.com/stac-utils/stac-geoparquet) file on R2,
using [rustac](https://github.com/stac-utils/rustac) — Python bindings to the
high-performance [stac-rs](https://github.com/stac-utils/stac-rs) Rust library.

Compared to the static-JSON approach (parsing JSON files with `pystac`),
this approach:
- Uses **DuckDB** under the hood for fast column-store queries
- Supports **spatial queries** via bounding box
- Supports **CQL2 filter expressions** for complex queries
- Scales to catalogs with thousands of items without downloading all JSON

## Install dependencies
```
pip install rustac xpystac
```

In [1]:
import rustac

PARQUET_URL = "https://r2-pub.openscicomp.io/stac/dynamical/catalog.parquet"

client = rustac.DuckdbClient()
print(f"rustac ready — querying {PARQUET_URL}")

rustac ready — querying https://r2-pub.openscicomp.io/stac/dynamical/catalog.parquet


## 1. List all items

In [2]:
items = list(client.search(PARQUET_URL, limit=100))
print(f"{len(items)} items in catalog:\n")
for item in items:
    props = item.get("properties", {})
    print(f"{item['id']}")
    print(f"  bbox:  {item.get('bbox')}")
    print(f"  start: {props.get('start_datetime')}")
    print(f"  end:   {props.get('end_datetime')}")
    print()

7 items in catalog:

ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  bbox:  [-180.0, -90.0, 180.0, 90.0]
  start: 2024-04-01T00:00:00+00:00
  end:   2026-02-23T00:00:00+00:00

noaa-gefs-analysis-v0-1-2
  bbox:  [-180.0, -90.0, 180.0, 90.0]
  start: 2000-01-01T00:00:00+00:00
  end:   2026-02-23T09:00:00+00:00

noaa-gefs-forecast-35-day-v0-2-0
  bbox:  [-180.0, -90.0, 180.0, 90.0]
  start: 2020-10-01T00:00:00+00:00
  end:   2026-02-23T00:00:00+00:00

noaa-gfs-analysis-v0-1-0
  bbox:  [-180.0, -90.0, 180.0, 90.0]
  start: 2021-05-01T00:00:00+00:00
  end:   2026-02-23T14:00:00+00:00

noaa-gfs-forecast-v0-2-7
  bbox:  [-180.0, -90.0, 180.0, 90.0]
  start: 2021-05-01T00:00:00+00:00
  end:   2026-02-23T06:00:00+00:00

noaa-hrrr-analysis-v0-1-0
  bbox:  [-134.12142793280145, 21.122192719272277, -60.891244531606546, 52.62870335266728]
  start: 2018-09-16T00:00:00+00:00
  end:   2026-02-23T14:00:00+00:00

noaa-hrrr-forecast-48-hour-v0-1-0
  bbox:  [-134.12142793280145, 21.122192719272277, -60

## 2. Spatial query — datasets that cover a region

Pass a `bbox` as `[lon_min, lat_min, lon_max, lat_max]`. DuckDB runs the
spatial intersection server-side.

In [3]:
italy_bbox = [6.6, 36.6, 18.5, 47.1]  # lon_min, lat_min, lon_max, lat_max

matches = list(client.search(PARQUET_URL, bbox=italy_bbox, limit=100))

print(f"Datasets covering Italy ({len(matches)} of {len(items)}):\n")
for item in matches:
    print(f"  {item['id']}")

Datasets covering Italy (5 of 7):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  noaa-gefs-analysis-v0-1-2
  noaa-gefs-forecast-35-day-v0-2-0
  noaa-gfs-analysis-v0-1-0
  noaa-gfs-forecast-v0-2-7


## 3. Temporal query — datasets covering a date range

For items with `start_datetime` / `end_datetime` (and `datetime: null`),
use a CQL2 `filter` expression to check interval overlap.

The overlap condition is:  `start_datetime <= query_end AND end_datetime >= query_start`

In [4]:
query_start = "2024-01-01T00:00:00Z"
query_end   = "2025-01-01T00:00:00Z"

time_filter = {
    "op": "and",
    "args": [
        {"op": "<=", "args": [{"property": "start_datetime"}, query_end]},
        {"op": ">=", "args": [{"property": "end_datetime"}, query_start]},
    ],
}

matches_time = list(client.search(PARQUET_URL, filter=time_filter, limit=100))

print(f"Datasets covering {query_start[:10]} – {query_end[:10]} ({len(matches_time)}):\n")
for item in matches_time:
    props = item.get("properties", {})
    print(f"  {item['id']}")
    print(f"    {props.get('start_datetime')} → {props.get('end_datetime')}")

Datasets covering 2024-01-01 – 2025-01-01 (7):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
    2024-04-01T00:00:00+00:00 → 2026-02-23T00:00:00+00:00
  noaa-gefs-analysis-v0-1-2
    2000-01-01T00:00:00+00:00 → 2026-02-23T09:00:00+00:00
  noaa-gefs-forecast-35-day-v0-2-0
    2020-10-01T00:00:00+00:00 → 2026-02-23T00:00:00+00:00
  noaa-gfs-analysis-v0-1-0
    2021-05-01T00:00:00+00:00 → 2026-02-23T14:00:00+00:00
  noaa-gfs-forecast-v0-2-7
    2021-05-01T00:00:00+00:00 → 2026-02-23T06:00:00+00:00
  noaa-hrrr-analysis-v0-1-0
    2018-09-16T00:00:00+00:00 → 2026-02-23T14:00:00+00:00
  noaa-hrrr-forecast-48-hour-v0-1-0
    2018-07-13T12:00:00+00:00 → 2026-02-23T12:00:00+00:00


## 4. Combined spatial + temporal query

Combine `bbox` with a CQL2 `filter` to find datasets that cover
both Italy **and** a given date range.

In [5]:
combined = list(client.search(
    PARQUET_URL,
    bbox=italy_bbox,
    filter=time_filter,
    limit=100,
))

print(f"Datasets covering Italy and {query_start[:10]} – {query_end[:10]} ({len(combined)}):\n")
for item in combined:
    print(f"  {item['id']}")

Datasets covering Italy and 2024-01-01 – 2025-01-01 (5):

  ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0
  noaa-gefs-analysis-v0-1-2
  noaa-gefs-forecast-35-day-v0-2-0
  noaa-gfs-analysis-v0-1-0
  noaa-gfs-forecast-v0-2-7


## 5. Open a matching dataset with xarray

rustac drops non-spec top-level attributes (like `storage:schemes`) when writing
GeoParquet, so we look the item up by ID in the original JSON catalog, which has
the full metadata needed by `xpystac` to open the icechunk store.

In [6]:
import pystac
import xarray as xr
import xpystac  # registers xarray backend for icechunk assets

CATALOG_URL = "https://r2-pub.openscicomp.io/stac/dynamical/catalog.json"

# Load the JSON catalog once (small — 7 items, ~1 KB)
catalog = pystac.Catalog.from_file(CATALOG_URL)
item_index = {item.id: item for item in catalog.get_items()}

# Pick the first match from the rustac query and fetch the full JSON item
match_id = combined[0]["id"]
item = item_index[match_id]
print(f"Opening: {item.id}")

asset_key = next(k for k in item.assets if "@" in k)
asset = item.assets[asset_key]

ds = xr.open_dataset(asset)
ds

Opening: ecmwf-ifs-ens-forecast-15-day-0-25-degree-v0-1-0


<xarray.Dataset> Size: 162TB
Dimensions:                                     (init_time: 694, lead_time: 85,
                                                 ensemble_member: 51,
                                                 latitude: 721, longitude: 1440)
Coordinates:
  * init_time                                   (init_time) datetime64[ns] 6kB ...
    ingested_forecast_length                    (init_time) timedelta64[ns] 6kB ...
    expected_forecast_length                    (init_time) timedelta64[ns] 6kB ...
  * lead_time                                   (lead_time) timedelta64[ns] 680B ...
    valid_time                                  (init_time, lead_time) datetime64[ns] 472kB ...
  * ensemble_member                             (ensemble_member) int16 102B ...
  * latitude                                    (latitude) float64 6kB 90.0 ....
  * longitude                                   (longitude) float64 12kB -180...
    spatial_ref                                 int64 8B ...
Data variables: (12/13)
    categorical_precipitation_type_surface      (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    downward_short_wave_radiation_flux_surface  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    downward_long_wave_radiation_flux_surface   (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    precipitation_surface                       (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    pressure_reduced_to_mean_sea_level          (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    dew_point_temperature_2m                    (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    ...                                          ...
    pressure_surface                            (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_u_10m                                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_v_100m                                 (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_u_100m                                 (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    wind_v_10m                                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
    geopotential_height_500hpa                  (init_time, lead_time, ensemble_member, latitude, longitude) float32 12TB ...
Attributes:
    dataset_id:           ecmwf-ifs-ens-forecast-15-day-0-25-degree
    dataset_version:      0.1.0
    name:                 ECMWF IFS ENS Forecast, 15 day, 0.25 degree
    description:          Ensemble weather forecasts from the ECMWF Integrate...
    attribution:          ECMWF IFS ENS Forecast data processed by dynamical....
    spatial_domain:       Global
    spatial_resolution:   0.25 degrees (~20km)
    time_domain:          Forecasts initialized 2024-04-01 00:00:00 UTC to Pr...
    time_resolution:      Forecasts initialized every 24 hours
    forecast_domain:      Forecast lead time 0-360 hours (0-15 days) ahead
    forecast_resolution:  Forecast step 0-144 hours: 3 hourly, 144-360 hours:...